In [1]:
import random, itertools, time, json
import pandas as pd
import matplotlib.pyplot as plt

random.seed(1)


In [2]:
def gen_uniform_random_3sat(n, m):
    clauses = []
    for _ in range(m):
        vars_ = random.sample(range(n), 3)
        clause = []
        for v in vars_:
            sign = random.choice([True, False])
            clause.append((v, sign))
        clauses.append(tuple(clause))
    return clauses

def eval_clause(clause, assignment):
    return any((assignment[v] if sign else (not assignment[v])) for v, sign in clause)

def num_satisfied(clauses, assignment):
    return sum(1 for c in clauses if eval_clause(c, assignment))

def instance_struct(clauses, n):
    occ = {i: [] for i in range(n)}
    for i, c in enumerate(clauses):
        for v, sign in c:
            occ[v].append((i, sign))
    return occ


In [3]:
def h_unsat(clauses, assignment):
    return sum(0 if eval_clause(c, assignment) else 1 for c in clauses)

def h_crit_score(clauses, assignment, occ):
    n = len(assignment)
    flip_gain = [0]*n
    for i, c in enumerate(clauses):
        if not eval_clause(c, assignment):
            for v, sign in c:
                would_satisfy = (not assignment[v]) if sign else assignment[v]
                if would_satisfy:
                    flip_gain[v] += 1
    return -max(flip_gain) if flip_gain else 0


In [4]:
def neighbors_flip1(assignment):
    n = len(assignment)
    for i in range(n):
        a = list(assignment)
        a[i] = not a[i]
        yield tuple(a)

def neighbors_sample_k(assignment, k, sample_limit=50):
    n = len(assignment)
    combos = list(itertools.combinations(range(n), k))
    random.shuffle(combos)
    for idx, combo in enumerate(combos[:sample_limit]):
        a = list(assignment)
        for i in combo: a[i] = not a[i]
        yield tuple(a)


In [5]:
def hill_climbing(clauses, n, occ, heuristic='unsat', max_iters=200, restarts=3):
    best_result = None
    for r in range(restarts):
        a = tuple(random.choice([False, True]) for _ in range(n))
        best_val = h_unsat(clauses,a) if heuristic=='unsat' else h_crit_score(clauses,a,occ)
        it = 0
        while it < max_iters and best_val > 0:
            it += 1
            improved = False
            best_nb = None
            best_nb_val = best_val
            for nb in neighbors_flip1(a):
                val = h_unsat(clauses,nb) if heuristic=='unsat' else h_crit_score(clauses,nb,occ)
                if val < best_nb_val:
                    best_nb_val = val
                    best_nb = nb
            if best_nb is not None:
                a = best_nb
                best_val = best_nb_val
                improved = True
            if not improved:
                break
        sat_count = num_satisfied(clauses, a)
        solved = (sat_count == len(clauses))
        if best_result is None or sat_count > best_result['sat_count']:
            best_result = {'assignment': a, 'sat_count': sat_count, 'solved': solved, 'iters': it, 'restarts': r+1}
        if solved:
            break
    return best_result


def beam_search(clauses, n, occ, beam_width=3, heuristic='unsat', max_iters=100):
    beam = [tuple(random.choice([False, True]) for _ in range(n)) for _ in range(beam_width)]
    seen = set(beam)
    best = None
    for it in range(max_iters):
        candidates = []
        for a in beam:
            sat = num_satisfied(clauses, a)
            if sat == len(clauses):
                return {'assignment': a, 'sat_count': sat, 'solved': True, 'iters': it}
            for nb in neighbors_flip1(a):
                if nb in seen: continue
                seen.add(nb)
                val = h_unsat(clauses,nb) if heuristic=='unsat' else h_crit_score(clauses,nb,occ)
                candidates.append((val, -num_satisfied(clauses, nb), nb))
        if not candidates:
            break
        candidates.sort()
        beam = [t[2] for t in candidates[:beam_width]]
        best_candidate = min(candidates)
        if best is None or -best_candidate[1] > best['sat_count']:
            best = {'assignment': best_candidate[2], 'sat_count': -best_candidate[1], 'solved': (-best_candidate[1]==len(clauses))}
    return best


def variable_neighborhood_descent(clauses, n, occ, heuristic='unsat', neighborhoods=[1,2,3], max_iters=200, restarts=2, sample_limit=50):
    best_result = None
    for r in range(restarts):
        a = tuple(random.choice([False, True]) for _ in range(n))
        current = a
        cur_val = h_unsat(clauses, current) if heuristic=='unsat' else h_crit_score(clauses, current, occ)
        it = 0
        k = 0
        while it < max_iters:
            neigh_size = neighborhoods[k]
            if neigh_size == 1:
                gen = neighbors_flip1(current)
            else:
                gen = neighbors_sample_k(current, neigh_size, sample_limit=sample_limit)
            best_nb = None
            best_nb_val = cur_val
            for nb in gen:
                it += 1
                val = h_unsat(clauses, nb) if heuristic=='unsat' else h_crit_score(clauses, nb, occ)
                if val < best_nb_val:
                    best_nb_val = val
                    best_nb = nb
            if best_nb is not None and best_nb_val < cur_val:
                current = best_nb
                cur_val = best_nb_val
                k = 0
            else:
                k += 1
                if k >= len(neighborhoods): break
            if cur_val == 0: break
        sat_count = num_satisfied(clauses, current)
        solved = (sat_count == len(clauses))
        if best_result is None or sat_count > best_result['sat_count']:
            best_result = {'assignment': current, 'sat_count': sat_count, 'solved': solved, 'iters': it}
        if solved:
            break
    return best_result


In [7]:
# --- Cell 7 (FIXED) ---
import time
import pandas as pd

n, m = 12, 36
clauses = gen_uniform_random_3sat(n, m)
occ = instance_struct(clauses, n)

# wrappers that accept the same keyword argument name
def beam3_wrapper(c, n_, occ_, heuristic='unsat'):
    return beam_search(c, n_, occ_, beam_width=3, heuristic=heuristic)

def beam4_wrapper(c, n_, occ_, heuristic='unsat'):
    return beam_search(c, n_, occ_, beam_width=4, heuristic=heuristic)

algorithms = [
    ("Hill Climb", hill_climbing),
    ("Beam (w=3)", beam3_wrapper),
    ("Beam (w=4)", beam4_wrapper),
    ("VND", variable_neighborhood_descent)
]

results = []
for name, algo in algorithms:
    for heuristic in ["unsat", "crit"]:
        t0 = time.time()
        out = algo(clauses, n, occ, heuristic=heuristic)
        t1 = time.time()

        # defensive handling in case algo returns None or missing keys
        if out is None:
            solved = False
            sat_count = 0
        else:
            solved = bool(out.get('solved', False))
            if 'sat_count' in out:
                sat_count = int(out['sat_count'])
            else:
                assignment = out.get('assignment')
                sat_count = num_satisfied(clauses, assignment) if assignment is not None else 0

        results.append({
            "Algorithm": name,
            "Heuristic": heuristic,
            "Solved": solved,
            "Satisfied": sat_count,
            "Total": len(clauses),
            "Time (s)": round(t1 - t0, 6)
        })

df = pd.DataFrame(results)
df



,Algorithm,Heuristic,Solved,Satisfied,Total,Time (s)
0,Hill Climb,unsat,True,36,36,0.001015
1,Hill Climb,crit,False,33,36,0.000000
2,Beam (w=3),unsat,True,36,36,0.002044
3,Beam (w=3),crit,False,33,36,0.141393
4,Beam (w=4),unsat,True,36,36,0.008163
5,Beam (w=4),crit,False,33,36,0.147776
6,VND,unsat,True,36,36,0.004288
7,VND,crit,False,30,36,0.009100
